In [110]:
import pandas as pd
import numpy as np


In [98]:
population_df = pd.read_csv("pop.csv")
life_expectancy_df = pd.read_csv("lex.csv")
gni_df = pd.read_csv("ny_gnp_pcap_pp_cd.csv")

In [99]:
print(population_df.info())
print(life_expectancy_df.info())
print(gni_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 197 entries, 0 to 196
Columns: 302 entries, country to 2100
dtypes: object(302)
memory usage: 464.9+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196 entries, 0 to 195
Columns: 302 entries, country to 2100
dtypes: float64(301), object(1)
memory usage: 462.6+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 198 entries, 0 to 197
Data columns (total 34 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   country  198 non-null    object
 1   1990     145 non-null    object
 2   1991     148 non-null    object
 3   1992     152 non-null    object
 4   1993     157 non-null    object
 5   1994     160 non-null    object
 6   1995     172 non-null    object
 7   1996     173 non-null    object
 8   1997     174 non-null    object
 9   1998     174 non-null    object
 10  1999     174 non-null    object
 11  2000     181 non-null    object
 12  2001     183 non-null    object
 13  2002 

# Fill missing Values

### population dataframe

In [100]:
# Set 'country' as the index to avoid filling it
population_df.set_index('country', inplace=True)

# Apply forward and backward fill across each row
population_df = population_df.apply(lambda row: row.ffill().bfill(), axis=1)

# Reset the index to restore the original DataFrame structure
population_df.reset_index(inplace=True)

In [101]:
# Set 'country' as the index to avoid filling it
life_expectancy_df.set_index('country', inplace=True)

# Apply forward and backward fill across each row
life_expectancy_df = life_expectancy_df.apply(lambda row: row.ffill().bfill(), axis=1)

# Reset the index to restore the original DataFrame structure
life_expectancy_df.reset_index(inplace=True)

In [102]:
# Set 'country' as the index to avoid filling it
gni_df.set_index('country', inplace=True)

# Apply forward and backward fill across each row
gni_df = gni_df.apply(lambda row: row.ffill().bfill(), axis=1)

# Reset the index to restore the original DataFrame structure
gni_df.reset_index(inplace=True)

In [103]:
print(population_df.head(1))
print(life_expectancy_df.head(1))
print(gni_df.head(1))

       country   1800   1801   1802   1803   1804   1805   1806   1807   1808  \
0  Afghanistan  3.28M  3.28M  3.28M  3.28M  3.28M  3.28M  3.28M  3.28M  3.28M   

   ...  2091  2092  2093  2094  2095  2096  2097  2098  2099  2100  
0  ...  108M  108M  109M  109M  109M  110M  110M  110M  111M  111M  

[1 rows x 302 columns]
       country  1800  1801  1802  1803  1804  1805  1806  1807  1808  ...  \
0  Afghanistan  28.2  28.2  28.2  28.2  28.2  28.2  28.1  28.1  28.1  ...   

   2091  2092  2093  2094  2095  2096  2097  2098  2099  2100  
0  75.9  76.1  76.2  76.4  76.5  76.7  76.8  77.0  77.1  77.3  

[1 rows x 302 columns]
  country   1990   1991   1992   1993   1994   1995   1996   1997   1998  ...  \
0   Aruba  20.5k  22.2k  23.2k  24.6k  26.3k  26.9k  26.8k  28.3k  28.6k  ...   

    2013   2014   2015   2016   2017 2018   2019   2020   2021   2022  
0  34.7k  35.5k  35.8k  35.9k  36.7k  39k  40.6k  34.1k  41.6k  41.6k  

[1 rows x 34 columns]


# Tidy Dataframe

Melt the dataframes: The melt function is used to transform the data into a tidy format. The id_vars parameter specifies the columns to keep as identifiers (in this case, country). The var_name parameter specifies the name of the new "variable" column (in this case, year), and the value_name parameter specifies the name of the new "value" column (in this case, population, life_expectancy, or gni_per_capita).

In [104]:
# Transform population data
population_tidy = population_df.melt(id_vars=['country'], var_name='year', value_name='population')

# Transform life expectancy data
life_expectancy_tidy = life_expectancy_df.melt(id_vars=['country'], var_name='year', value_name='life_expectancy')

# Transform GNI per capita data
gni_per_capita_tidy = gni_df.melt(id_vars=['country'], var_name='year', value_name='gni')

In [105]:
# Check the first few rows of each tidy dataframe
print(population_tidy.head())
print(life_expectancy_tidy.head())
print(gni_per_capita_tidy.head())

       country  year population
0  Afghanistan  1800      3.28M
1       Angola  1800      1.57M
2      Albania  1800       400k
3      Andorra  1800       2650
4          UAE  1800      40.2k
       country  year  life_expectancy
0  Afghanistan  1800             28.2
1       Angola  1800             27.0
2      Albania  1800             35.4
3      Andorra  1800             74.6
4          UAE  1800             30.7
       country  year    gni
0        Aruba  1990  20.5k
1  Afghanistan  1990   1570
2       Angola  1990   3000
3      Albania  1990   2550
4          UAE  1990  98.6k


# Merge Dataframes

In [106]:
merged_df = (population_tidy
             .merge(life_expectancy_tidy, on=['country', 'year'], how='outer')
             .merge(gni_per_capita_tidy, on=['country', 'year'], how='outer'))

In [107]:
def normalize_value(value):
    """
    Converts a string representation of a number with potential suffixes ('k', 'M', 'B')
    to an integer.
    """
    if isinstance(value, str):
        if 'k' in value:
            return int(float(value.replace('k', '')) * 1e3)
        elif 'M' in value:
            return int(float(value.replace('M', '')) * 1e6)
        elif 'B' in value:
            return int(float(value.replace('B', '')) * 1e9)
        else:
            return int(value)
    return value

In [108]:
for column in ['population', 'gni']:
    merged_df[column] = merged_df[column].apply(normalize_value)

In [111]:

merged_df['gni'] = merged_df['gni'].replace(0, np.nan)  # Replace zeros with NaN to avoid log(0)
merged_df['gni'] = np.log(merged_df['gni']) 

In [112]:
print(merged_df.head())

       country  year  population  life_expectancy  gni
0  Afghanistan  1800   3280000.0             28.2  NaN
1       Angola  1800   1570000.0             27.0  NaN
2      Albania  1800    400000.0             35.4  NaN
3      Andorra  1800      2650.0             74.6  NaN
4          UAE  1800     40200.0             30.7  NaN
